---

## 📋 PydanticOutputParser 핵심 이해하기

`PydanticOutputParser`는 언어 모델의 출력을 **구조화된 정보**로 변환하는 데 도움을 주는 클래스입니다. 마치 **자동 분류기계**처럼 단순 텍스트 응답 대신 **명확하고 체계적인 형태로 필요한 정보를 제공**할 수 있습니다. 🏭

### 🔧 PydanticOutputParser의 핵심 메서드

`PydanticOutputParser`에는 **두 가지 핵심 메서드**가 구현되어야 합니다:

#### 1️⃣ **`get_format_instructions()`** - 📝 작성 가이드 제공
- **역할**: 언어 모델이 출력해야 할 정보의 **형식을 정의하는 지침** 제공
- **예시**: "이메일에서 발신자, 제목, 날짜를 JSON 형태로 추출해주세요"
- **중요성**: 이 지침이 명확해야 AI가 원하는 형태로 답변을 생성함

#### 2️⃣ **`parse()`** - 🔄 텍스트를 데이터로 변환
- **역할**: AI 모델의 출력(문자열)을 **특정 구조로 분석하고 변환**
- **과정**: 입력된 문자열 → 검증 → 사전 정의된 스키마에 맞는 데이터 구조로 변환
- **도구**: Pydantic을 사용한 타입 안전성 보장

### 🏗️ 왜 이런 구조로 나눠놨을까?

```
📝 get_format_instructions() → AI에게 "이렇게 답변해줘" 라고 설명
                                   ↓
🤖 AI 모델이 지침에 따라 구조화된 텍스트 생성
                                   ↓  
🔄 parse() → "생성된 텍스트를 실제 데이터로 변환"
```

### 📚 참고 자료

- [Pydantic 공식 도큐먼트](https://docs.pydantic.dev/latest/) - 데이터 검증 및 설정 관리 라이브러리
- [LangChain OutputParsers](https://python.langchain.com/docs/modules/model_io/output_parsers/) - 다양한 출력 파서들

---

## 🛠️ 환경 설정

PydanticOutputParser를 사용하기 전에 필요한 도구들을 준비해봅시다!

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

In [ ]:
# 실시간 출력을 위한 import (스트리밍 응답을 보기 좋게 출력)
from langchain_teddynote.messages import stream_response

In [ ]:
# 필요한 라이브러리 임포트
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(temperature=0, model_name="gpt-4.1")

---

## 📧 실습 데이터 준비하기

이제 실제로 **PydanticOutputParser**의 능력을 테스트해볼 **실습용 이메일 데이터**를 준비해봅시다! 

### 🎯 실습 시나리오

아래는 **비즈니스 협력 제안 이메일**입니다. 이 이메일에서 다음과 같은 **핵심 정보들**을 자동으로 추출해보겠습니다:

- **👤 발신자 정보** (이름, 이메일)
- **📋 이메일 제목**  
- **📝 내용 요약**
- **🗓️ 미팅 일정**

실제 업무에서 이런 이메일이 수십, 수백 개씩 들어온다면? **PydanticOutputParser**가 자동으로 정리해줄 수 있습니다! 🚀

In [ ]:
# 실습용 이메일 데이터 (비즈니스 협력 제안 메일 예시)
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

---

## 🔄 비교 실험: 파서 없이 vs 파서와 함께

먼저 **PydanticOutputParser를 사용하지 않았을 때**의 결과를 확인해봅시다.

### 🤖 일반적인 AI 응답 (구조화되지 않음)

파서 없이 AI에게 이메일 정보 추출을 요청하면 어떤 결과가 나올까요?

In [ ]:
# 출력 파서 없이 기본 체인 구성
from itertools import chain
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\\n\\n{email_conversation}"
)

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(temperature=0, model_name="gpt-4.1")

# 체인 구성 (프롬프트 + 모델)
chain = prompt | llm

# 체인 실행 (스트리밍 방식)
answer = chain.stream({"email_conversation": email_conversation})

# 실시간 출력 및 결과 반환
output = stream_response(answer, return_output=True)

In [ ]:
# 파서 없이 실행한 결과 출력 (구조화되지 않은 텍스트 형태)
print(output)

### 📋 결과 분석

위에서 본 것처럼 파서 없이는 **자연스러운 문장 형태**로 정보가 제공됩니다. 사람이 읽기에는 좋지만, **프로그램이 처리하기에는 불편**합니다.

---

## 🏗️ Pydantic 모델로 데이터 구조 설계하기

이제 **체계적인 데이터 구조**를 만들어봅시다! 

### 🎯 EmailSummary 모델 설계

아래의 **Pydantic 모델**을 사용하여 이메일의 정보를 **구조화된 형태**로 파싱해 보겠습니다.

#### 🔑 Field description의 중요성

`Field` 안의 `description`은 **AI에게 주는 명령서**입니다! 

- **🎯 정확성**: 설명이 명확할수록 AI가 정확한 정보를 추출
- **🧭 가이드 역할**: "어떤 정보를 찾아야 하는지" AI에게 알려줌  
- **🔍 필터링**: 불필요한 정보는 제외하고 핵심만 추출

> 💡 **팁**: description은 마치 **"동료에게 업무를 설명하는 것"**처럼 구체적이고 명확하게 작성하세요!

In [ ]:
# 이메일 정보를 구조화하기 위한 Pydantic 모델 정의
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 인스턴스 생성 (EmailSummary 모델 기반)
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [ ]:
# AI 모델에게 전달할 형식 지침 확인 (JSON 스키마 형태로 생성됨)
print(parser.get_format_instructions())

### 🔍 Format Instructions 살펴보기

위에서 출력된 내용이 바로 **AI에게 전달되는 작업 지침서**입니다! 

- **📋 JSON 스키마**: 어떤 형태로 답변해야 하는지 명시
- **📝 필드 설명**: 각 필드에 어떤 정보를 넣어야 하는지 설명
- **⚠️ 주의사항**: JSON 형식을 정확히 지켜야 한다는 안내

---

## 📝 프롬프트 템플릿 구성하기

이제 **AI에게 정확한 지시**를 내리기 위한 프롬프트를 만들어봅시다!

### 🎯 프롬프트 구성 요소

1. **`question`**: 사용자의 구체적인 요청사항
2. **`email_conversation`**: 분석할 이메일 본문 내용  
3. **`format`**: 출력 형식 지침 (자동으로 삽입됨)

### 💡 `partial()` 함수의 역할

`partial(format=parser.get_format_instructions())`는 **미리 format 필드를 채워놓는** 기능입니다:

```python
# 실행 전: {question}, {email_conversation}, {format}
# partial 적용 후: {question}, {email_conversation}, [형식 지침이 이미 채워짐]
```

이렇게 하면 나중에 실행할 때 **question과 email_conversation만 입력**하면 됩니다! 🚀

In [ ]:
# 구조화된 출력을 위한 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template(
    """You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# 프롬프트에 파서의 형식 지침을 부분적으로 적용 (미리 format 필드를 채워놓음)
prompt = prompt.partial(format=parser.get_format_instructions())

---

## 🔗 체인 구성하기

이제 **프롬프트와 AI 모델을 연결**해봅시다! 

### 🚀 LangChain의 파이프라인 (`|` 연산자)

```python
chain = prompt | llm
```

이것은 마치 **공장의 컨베이어 벨트**와 같습니다:

```
📝 프롬프트 (지시서 작성) → 🤖 AI 모델 (작업 수행) → 📄 결과 출력
```

### 💡 왜 체인으로 연결할까?

- **🔄 재사용성**: 한 번 만들어놓으면 여러 번 사용 가능
- **🧩 모듈성**: 각 단계를 독립적으로 교체/수정 가능  
- **📊 추적성**: 각 단계별 결과를 확인하기 쉬움

In [ ]:
# 프롬프트와 모델을 연결한 체인 생성 (파서는 별도로 적용 예정)
chain = prompt | llm

---

## ⚡ 체인 실행하고 결과 확인하기

이제 우리가 만든 체인을 **실제로 실행**해봅시다!

### 📊 실행 결과의 특징

- **📝 자연스러운 문장**: 사람이 읽기에는 좋음
- **🤖 JSON 형태 시도**: AI가 구조화하려고 노력함  
- **⚠️ 파싱 필요**: 아직 프로그램이 직접 사용하기는 어려움

여전히 **텍스트 형태의 응답**이므로, 이것을 **실제 데이터 객체**로 변환해야 합니다!

In [ ]:
# 체인 실행 (스트리밍 방식으로 응답 생성)
response = chain.stream(
    {
        "email_conversation": email_conversation,  # 이메일 내용 입력
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",  # 사용자 질문
    }
)

# 스트리밍 응답을 실시간으로 출력하고 결과 저장
output = stream_response(response, return_output=True)

---

## 🔄 마법의 변환: 텍스트 → 구조화된 데이터

이제 **PydanticOutputParser의 진짜 능력**을 확인해봅시다!

### 🎯 `parse()` 메서드의 역할

```python
parser.parse(output)  # 텍스트 → EmailSummary 객체
```

이 과정에서 일어나는 일들:

1. **📝 텍스트 분석**: JSON 형태의 문자열을 파싱
2. **✅ 데이터 검증**: Pydantic이 타입과 형식을 검증  
3. **🏗️ 객체 생성**: EmailSummary 인스턴스로 변환
4. **⚡ 속성 접근 가능**: `result.person`, `result.email` 등으로 접근

### 🎉 결과 확인

이제 **점 표기법(dot notation)**으로 각 필드에 바로 접근할 수 있습니다!

In [ ]:
# 텍스트 응답을 EmailSummary 객체로 변환 (구조화된 데이터로 파싱)
structured_output = parser.parse(output)
print(structured_output)

---

## 🚀 완전한 파이프라인: 원스톱 처리

이제 **모든 과정을 하나로 연결**해봅시다!

### 🔗 3단계 파이프라인

```python
chain = prompt | llm | parser
```

이렇게 하면 **한 번의 실행**으로 모든 과정이 자동으로 처리됩니다:

```
📝 프롬프트 → 🤖 AI 모델 → 🔄 파서 → ✨ 구조화된 객체
```

### 🎯 장점

- **⚡ 효율성**: 한 번의 호출로 모든 처리 완료
- **🛡️ 안정성**: 각 단계가 자동으로 연결되어 오류 최소화
- **🧹 깔끔함**: 중간 결과물을 저장할 필요 없음

### 💡 결과물의 특징

출력 결과를 정의한 **Pydantic 객체로 자동 생성**되므로 바로 프로그램에서 활용할 수 있습니다!

In [ ]:
# 파서까지 포함한 완전한 체인 구성 (프롬프트 + 모델 + 파서)
chain = prompt | llm | parser

In [ ]:
# 완전한 체인 실행 (한 번에 구조화된 결과 생성)
response = chain.invoke(
    {
        "email_conversation": email_conversation,  # 이메일 내용 입력
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",  # 사용자 질문
    }
)

# 결과가 자동으로 EmailSummary 객체 형태로 반환됨
response

---

## 🚀 더 간편한 방법: `with_structured_output()`

**PydanticOutputParser**보다 **더 간단한 방법**이 있습니다!

### 🎯 `with_structured_output()`의 장점

이 방법은 **프롬프트 템플릿이나 별도 파서 없이** 바로 구조화된 출력을 받을 수 있습니다!

#### 🆚 비교: PydanticOutputParser vs with_structured_output()

| 특징 | PydanticOutputParser | with_structured_output() |
|------|---------------------|-------------------------|
| **🛠️ 설정 복잡도** | 높음 (프롬프트 + 파서 필요) | 낮음 (모델에 직접 적용) |
| **🔧 커스터마이징** | 높음 (세부 제어 가능) | 중간 (기본 설정 위주) |
| **⚡ 실행 속도** | 약간 느림 | 빠름 |
| **🎛️ 프롬프트 제어** | 완전 제어 가능 | 제한적 |

### 💡 언제 어떤 것을 사용할까?

- **🏃 빠른 프로토타입**: `with_structured_output()` 추천
- **🔧 세밀한 제어**: `PydanticOutputParser` 추천
- **📚 학습 목적**: 두 방법 모두 이해하면 좋음

In [ ]:
# with_structured_output을 사용한 구조화된 출력
llm_with_structered = ChatOpenAI(
    temperature=0, model_name="gpt-4.1"
).with_structured_output(EmailSummary)

In [ ]:
# with_structured_output을 사용한 간편한 구조화된 출력
answer = llm_with_structered.invoke(email_conversation)
answer

In [ ]:
# invoke() 함수를 호출하여 결과를 출력합니다.
answer = llm_with_structered.invoke(email_conversation)
answer

**참고**

한 가지 아쉬운 점은 `.with_structured_output()` 함수는 `stream()` 기능을 지원하지 않습니다.